In [1]:
using Plots

# Solutions for Radial Schrodinger Equation for the Neutron-Proton System
### Gabe Schumm
### Due October 26th, 2023

### Numerov's Algorithm
$U ''(x) = f(x)\Psi (x) \longrightarrow U ''_k = f_kU_k$


$\phi_k = U_k\left(1 - \frac{\left(\Delta r\right)^2 f_k}{12}\right)$

$\phi_{k+1} = 2\phi_k - \phi_{k-1} + \left(\Delta r\right)^2f_kU_k$

In [2]:
#module containg struct for this program
module Program 
    mutable struct System
        β::Float64 #energy parameter in fm^-2
        E::Float64 #energy value in MeV

        r_0::Float64 #minimum radial distance, U(r_0) = 0 in fm
        a::Float64 #range parameters of Yukawa potential in fm
        V_0::Float64 #streng of Yukawa potential in units of E

        r_max::Float64 #maximum radial distance for integration
        ∆r::Float64 #radial distance step
       
        n_total::Int64 #total number of integrations steps, determined by r_max - r_0 and ∆r
        
        r::Float64 #current position
        #current values of phi_n
        phi_0::Float64 #
        phi_1::Float64
        phi_2::Float64
        
        U::Vector{Float64} #U(r) wavefunction

    end

    function System(r_0, a, V_0, r_max, ∆r)
        β = -0.053667
        E = -2.226 
        n_total = round(Int64, (r_max - r_0) / ∆r) + 1 #n_total must be odd for Simpsons rule integration
        U = zeros(n_total)
        System(β, E, r_0, a, V_0, r_max, ∆r, n_total, r_max, 0,0,0, U)
    end

end

System = Program.System



Main.Program.System

In [3]:
#Yukawa Potential
function V_Y(self, r)
    return -self.V_0*abs(self.E) * exp(-r/self.a) * (self.a/r)
end

#Asumptotic form of U
function U_asymp(self, r)
    return exp(-sqrt(-self.β)*r)
end

#f function in second order diff. eq. for U(r)
function f(self, r)
    return self.β * ((V_Y(self, r)/self.E) - 1)
end

f (generic function with 1 method)

In [4]:
function initialize(self)
    
    #reset U, phi_2
    self.U = zeros(self.n_total)
    self.phi_2 = 0
    
    #set value of U at r = r_max first
    self.U[self.n_total] = U_asymp(self, self.r_max)
    #set corresponding vaue of phi_0
    self.phi_0 = self.U[self.n_total] * (1-((self.∆r^2) * f(self, self.r_max) / 12))
    
    #set value of U at r = r_max - ∆r next
    self.U[self.n_total-1] = U_asymp(self, self.r_max - self.∆r)
    #set corresponding vaue of phi_1
    self.phi_1 = self.U[self.n_total-1] * (1-((self.∆r^2) * f(self, self.r_max - self.∆r) / 12))
    
    #set radius for third integration step 
    self.r = self.r_max - self.∆r
    
end

initialize (generic function with 1 method)

In [5]:
#integrate U(r) from r_max to r_0
function U_integration(self)
    #initial f(r) value
    f1 = f(self, self.r)
    for n=1:self.n_total-2
        
        #calculate phi_2 using phi_1, phi_0, U_1
        self.phi_2 = 2*self.phi_1 - self.phi_0 + ((self.∆r)^2 * f1 * self.U[self.n_total-n])
        self.phi_0 = self.phi_1; self.phi_1 = self.phi_2
        
        #progress radial position
        self.r -= self.∆r
        
        #calculate f for next integration step
        f1 = f(self, self.r)
        #store value of U(r) using recently updated phi_1 and position 
        
        self.U[self.n_total-n-1] = self.phi_1/(1-((self.∆r)^2 * f1 / 12))
        
    end
end

U_integration (generic function with 1 method)

In [10]:
#locate V_0 that satisfies boundary condition for given value of a
function find_V0(self, ∆V, ϵ)
    
    #start with integration using initial V_0 and save U(r_0) as prev_U0
    initialize(self)
    U_integration(self)
    prev_U0 = self.U[1]
    
    #changes to true when U(r_0) changes from positive/negative to negative/positive after changing V_0
    pass_zero = false
    
    #increment V_0 by ∆V
    self.V_0 += ∆V
    
    println("Locating Initial V_0.")
    flush(stdout)
    println()
    
    while !pass_zero #increase V_0 until pass_zero condition satisfied
        
        initialize(self)
        U_integration(self)
        
        #pass through zero condition for previous integration and current integration
        if self.U[1]*prev_U0 < 0
            #if true, exit while loop
            pass_zero = true
        else
            #if not true, store U(r_0) and increment V_0
            prev_U0 = self.U[1]
            self.V_0 += ∆V
        end
    end
    
    println("Initial V_0/|E| = ", self.V_0)
    flush(stdout)
    
   
    
    #once pass through zero V_0 is found, start bisection routine
    V_1 = self.V_0 - ∆V
    V_2 = self.V_0
    self.V_0 = (V_1 + V_2)/2
    
    println("Starting Bisection.")
    println()
    
    #track how many bisection steps taken
    count = 0
    
    #run bisection rountine until change in U(r_0) is below given threshold ϵ
    while abs(prev_U0-self.U[1]) > ϵ
    #while abs(V_1-V_2) > ϵ

        #store current U(r_0)
        prev_U0 = self.U[1]
        
        #print count, V_0 bracket, and U(r_0)
        println(count,". ",V_1, " ",V_2," ",self.V_0, " ",prev_U0)
        flush(stdout)
        count+=1
        
        initialize(self)
        U_integration(self)
        
        #if U(r_0) passes through zero, update lower V_0 bracket 
        if self.U[1]*prev_U0 < 0
            V_1 = self.V_0
            self.V_0 = (V_1 + V_2)/2
        
        #otherwise update upper V_0 bracket
        else
            V_2 = self.V_0
            self.V_0 = (V_1 + V_2)/2
        end
    end
    
    println()
    println("After ", count, " steps, convergent value found.")
    println("U(r_0) = ", self.U[1], " for V_0/|E| = ", self.V_0)
    flush(stdout)
end

find_V0 (generic function with 1 method)

In [11]:
#calculate norm of R(r)^2 = U(r)/r by integrating from r_0 to r_max using Simpsons rule
#note: this is integration in spherical coordinates so factor of r^2 must be included from the in Jacobian
#|R(r)|^2 * (r^2 dr) =  |U(r)|^2 dr
function calculate_norm(self)
    #first term has weight of 1
    I = (self.U[1])^2
    power = 2
    for n=2:self.n_total-1
        #second term has weight of 4 = 2^2 and third term has weight of 2 = 2^1
        I += 2^power * (self.U[n])^2
        #recursive relation for p in 2^p gives 4, 2, 4, 2... pattern
        power = 3 - power
    end
    #last term has weight of 1
    I += (self.U[end])^2
    
    #multiply by intgration step divided by 3
    return (self.∆r/3) * I
end


#calculate sqrt(1/4<r^2>) using Simpsons rule
#r^2 * |R(r)|^2 * (r^2 dr) = r^2 * |U(r)|^2 dr
function calculate_r(self)
    norm = calculate_norm(self)
    
    I = (self.U[1] * self.r_0)^2 
    power = 2
    for n=2:self.n_total-1
        #multiply by current r value
        I += 2^power * (self.U[n]*(self.r_0 + (n-1)*self.∆r))^2
        power = 3 - power
    end
    #println(power)
    I += (self.U[end] * self.r_max)^2
    I *= (self.∆r/3) / norm #divide by norm of R(r)
    return sqrt(I/4)
end


calculate_r (generic function with 1 method)

In [12]:
#routine to locate optimal V_0 for a given value of a
#export a, V_0, √<r^2> to outfile f
function export_r_V0(a, ∆r, r_max, file)
    #start with "quick and dirty" run starting at V_0 = 0
    
    self = System(.1, a, 0, r_max, ∆r)
    initialize(self)
    find_V0(self, 1, .0001) #use larger ∆V and large cutoff ϵto get in the neighborhood of the true V_0
    
    println()
    
    #repeat with initial V_0 guess near the V_0 found above
    V_0 = self.V_0-1 #ensure initial V_0 is less than optimal V_0
    
    self = System(.1, a, V_0, r_max, ∆r)
    initialize(self)
    find_V0(self, .001, .00000001) #use smaller ∆V and smaller cutoff ϵ 
    
    #calculate mean value of 
    mean_r = calculate_r(self)
    
    println(file, a, " ", round(mean_r, digits=5), " ", round(self.V_0, digits=5))
    return
end

export_r_V0 (generic function with 1 method)

## Find $(a, V_0)$ pairs for given values of $a$

In [9]:
a_values = .5:.1:3

open("vr.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 30, file)
    end
end

Locating Initial V_0.

Initial V_0/|E| = 237.0
Starting Bisection.

0. 236.0 237.0 236.5 -0.0012389610737520112
1. 236.0 236.5 236.25 -0.00015402960848935756
2. 236.25 236.5 236.375 0.00038963094477489186
3. 236.25 236.375 236.3125 0.0001177010312898666
4. 236.25 236.3125 236.28125 0.0002536410736933635

After 5 steps, convergent value found.
U(r_0) = 0.0003216297799901303 for V_0/|E| = 236.265625

Locating Initial V_0.

Initial V_0/|E| = 236.42962500000556
Starting Bisection.

0. 236.42862500000555 236.42962500000556 236.42912500000557 -1.0697671886619866e-6
1. 236.42912500000557 236.42962500000556 236.42937500000556 1.720712739994327e-8
2. 236.42937500000556 236.42962500000556 236.42950000000556 -5.262804529479043e-7
3. 236.42937500000556 236.42950000000556 236.42943750000558 -7.980239680902179e-7
4. 236.42937500000556 236.42943750000558 236.42940625000557 -6.621522003102255e-7
5. 236.42937500000556 236.42940625000557 236.42939062500557 -5.94216320247077e-7
6. 236.42937500000556 236.

Locating Initial V_0.

Initial V_0/|E| = 46.0
Starting Bisection.

0. 45.0 46.0 45.5 -0.00869396031123247
1. 45.0 45.5 45.25 -0.0037072839906535024
2. 45.0 45.25 45.125 -0.0011795361014060694
3. 45.125 45.25 45.1875 9.299241593169655e-5
4. 45.1875 45.25 45.21875 -0.0005439946136648989
5. 45.1875 45.21875 45.203125 -0.0008619459333814993
6. 45.1875 45.203125 45.1953125 -0.0007030154320812497

After 7 steps, convergent value found.
U(r_0) = -0.0006235163143091384 for V_0/|E| = 45.19140625

Locating Initial V_0.

Initial V_0/|E| = 45.1344062499978
Starting Bisection.

0. 45.133406249997805 45.1344062499978 45.13390624999781 -2.9666118362325968e-6
1. 45.13390624999781 45.1344062499978 45.1341562499978 2.1333754183607612e-6
2. 45.1341562499978 45.1344062499978 45.1342812499978 -4.166297551129807e-7
3. 45.1341562499978 45.1342812499978 45.1342187499978 -1.6916236912181793e-6
4. 45.1341562499978 45.1342187499978 45.1341874999978 -1.054127469426855e-6
5. 45.1341562499978 45.1341874999978 45.13

7. 22.125 22.1328125 22.12890625 0.0019233086622469403

After 8 steps, convergent value found.
U(r_0) = 0.0019991000596601563 for V_0/|E| = 22.126953125

Locating Initial V_0.

Initial V_0/|E| = 22.23295312500135
Starting Bisection.

0. 22.23195312500135 22.23295312500135 22.23245312500135 -1.1498991744821913e-5
1. 22.23195312500135 22.23245312500135 22.232203125001348 -1.8775913415364495e-6
2. 22.232203125001348 22.23245312500135 22.23232812500135 2.9332556126365743e-6
3. 22.232203125001348 22.23232812500135 22.232265625001347 5.278198850157063e-7
4. 22.232203125001348 22.232265625001347 22.232234375001347 1.7305347187026093e-6
5. 22.232203125001348 22.232234375001347 22.232218750001348 2.3318943470836335e-6
6. 22.232203125001348 22.232218750001348 22.23221093750135 2.632574855986222e-6
7. 22.232203125001348 22.23221093750135 22.23220703125135 2.782915131785639e-6
8. 22.232203125001348 22.23220703125135 22.23220507812635 2.8580854034316347e-6
9. 22.232203125001348 22.23220507812635 22

2. 13.75 14.0 13.875 -0.0016004726392254737
3. 13.75 13.875 13.8125 -0.005200124244863601
4. 13.75 13.8125 13.78125 -0.0034083975853466415
5. 13.75 13.78125 13.765625 -0.0025064651537831354
6. 13.75 13.765625 13.7578125 -0.002053977064936429
7. 13.75 13.7578125 13.75390625 -0.001827351976518818
8. 13.75 13.75390625 13.751953125 -0.0017139440992741102

After 9 steps, convergent value found.
U(r_0) = -0.0016572163184180224 for V_0/|E| = 13.7509765625

Locating Initial V_0.

Initial V_0/|E| = 13.695976562499476
Starting Bisection.

0. 13.694976562499477 13.695976562499476 13.695476562499476 -2.4630813028671734e-5
1. 13.694976562499477 13.695476562499476 13.695226562499476 -9.989026854584417e-6
2. 13.694976562499477 13.695226562499476 13.695101562499477 -2.667741277331015e-6
3. 13.695101562499477 13.695226562499476 13.695164062499476 9.929997057677764e-7
4. 13.695164062499476 13.695226562499476 13.695195312499475 -8.37378940512093e-7
5. 13.695164062499476 13.695195312499475 13.695179687499

8. 10.164976562499458 10.164980468749459 10.164978515624458 1.1587617092505792e-5
9. 10.164976562499458 10.164978515624458 10.164977539061958 1.1661126707577997e-5
10. 10.164976562499458 10.164977539061958 10.164977050780708 1.1697881570853774e-5
11. 10.164976562499458 10.164977050780708 10.164976806640084 1.171625900208613e-5

After 12 steps, convergent value found.
U(r_0) = 1.17254477273131e-5 for V_0/|E| = 10.16497668456977
Locating Initial V_0.

Initial V_0/|E| = 10.0
Starting Bisection.

0. 9.0 10.0 9.5 -0.018134393155727763
1. 9.5 10.0 9.75 0.0007444962428795427
2. 9.75 10.0 9.875 -0.008960179065520792
3. 9.75 9.875 9.8125 -0.01361252504895846
4. 9.75 9.8125 9.78125 -0.011302794400382564
5. 9.75 9.78125 9.765625 -0.010135613980328256
6. 9.75 9.765625 9.7578125 -0.009548930422751432
7. 9.75 9.7578125 9.75390625 -0.009254813480445915
8. 9.75 9.75390625 9.751953125 -0.009107560989735421

After 9 steps, convergent value found.
U(r_0) = -0.00903388621094351 for V_0/|E| = 9.7509765625


1. 7.519476562500257 7.519976562500257 7.5197265625002565 1.4578525855968806e-5
2. 7.519476562500257 7.5197265625002565 7.519601562500257 2.5378593551019653e-6
3. 7.519476562500257 7.519601562500257 7.519539062500257 8.558082730201755e-6
4. 7.519476562500257 7.519539062500257 7.519507812500257 1.156827682372279e-5
5. 7.519476562500257 7.519507812500257 7.519492187500257 1.3073394432935547e-5
6. 7.519476562500257 7.519492187500257 7.519484375000257 1.3825958416506791e-5
7. 7.519476562500257 7.519484375000257 7.519480468750257 1.4202241651826584e-5
8. 7.519476562500257 7.519480468750257 7.5194785156252575 1.4390383609074901e-5
9. 7.519476562500257 7.5194785156252575 7.519477539062757 1.4484454737646153e-5
10. 7.519476562500257 7.519477539062757 7.519477050781507 1.4531490256516988e-5
11. 7.519476562500257 7.519477050781507 7.519476806640882 1.4555008044977826e-5
12. 7.519476562500257 7.519476806640882 7.51947668457057 1.4566766897104532e-5

After 13 steps, convergent value found.
U(r_0) 

## Testing $\Delta_r$  

### Comparing $V_0$ vs. $a$

In [ ]:
a_values = .5:.1:3

open("vr_.0001.dat", "w") do file
    for a in a_values
        export_r_V0(a, .0001, 30, file)
    end
end

open("vr_.001.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 30, file)
    end
end

open("vr_.01.dat", "w") do file
    for a in a_values
        export_r_V0(a, .01, 30, file)
    end
end

open("vr_.1.dat", "w") do file
    for a in a_values
        export_r_V0(a, .1, 30, file)
    end
end

### Comparing $U(r)$ vs. $r$

In [229]:
a = 1.3
V_0 = 32.7212

self_10_4 = System(.1, a, V_0, 30, .0001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_10_4)
U_integration(self_10_4)

r_10_4 = collect(self_10_4.r_0:self_10_4.∆r:self_10_4.r_max)

open("U_deltar_.0001.dat", "w") do file
    for i=1:length(r_10_4)
        println(file, r_10_4[i], " ", self_10_4.U[i])
    end
end

self_10_3 = System(.1, a, V_0, 30, .001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_10_3)
U_integration(self_10_3)

r_10_3 = collect(self_10_3.r_0:self_10_3.∆r:self_10_3.r_max)

open("U_deltar_.001.dat", "w") do file
    for i=1:length(r_10_3)
        println(file, r_10_3[i], " ", self_10_3.U[i])
    end
end


self_10_2 = System(.1, a, V_0, 30, .01)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_10_2)
U_integration(self_10_2)

r_10_2 = collect(self_10_2.r_0:self_10_2.∆r:self_10_2.r_max)

open("U_deltar_.01.dat", "w") do file
    for i=1:length(r_10_2)
        println(file, r_10_2[i], " ", self_10_2.U[i])
    end
end


0.0001
0.001
0.01


## Testing $r_{max}$

### Comparing $V_0$ vs. $a$

In [240]:
a_values = .5:.1:3

open("vr_50.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 50, file)
    end
end

open("vr_40.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 40, file)
    end
end

open("vr_30.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 30, file)
    end
end
open("vr_20.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 20, file)
    end
end

open("vr_10.dat", "w") do file
    for a in a_values
        export_r_V0(a, .001, 10, file)
    end
end


Locating Initial V_0.

Initial V_0/|E| = 237.0
Starting Bisection.

0. 236.0 237.0 236.5 -0.001239066171607346
1. 236.0 236.5 236.25 -0.0001541341813195194
2. 236.25 236.5 236.375 0.0003895266352555894
3. 236.25 236.375 236.3125 0.00011759659039392205
4. 236.25 236.3125 236.28125 0.0002535366982126211

After 5 steps, convergent value found.
U(r_0) = 0.00032152543673105786 for V_0/|E| = 236.265625

Locating Initial V_0.

Initial V_0/|E| = 236.42962500000556
Starting Bisection.

0. 236.42862500000555 236.42962500000556 236.42912500000557 -1.1742649582963352e-6
1. 236.42862500000555 236.42912500000557 236.42887500000558 -8.729101753827695e-8
2. 236.42887500000558 236.42912500000557 236.42900000000557 4.5619847410678215e-7
3. 236.42887500000558 236.42900000000557 236.4289375000056 1.8445228417704231e-7
4. 236.42887500000558 236.4289375000056 236.42890625000558 3.20324837695221e-7
5. 236.42887500000558 236.42890625000558 236.42889062500558 3.882621658309898e-7
6. 236.42887500000558 236.4288

Locating Initial V_0.

Initial V_0/|E| = 46.0
Starting Bisection.

0. 45.0 46.0 45.5 -0.008693984695477861
1. 45.0 45.5 45.25 -0.00370730736507077
2. 45.0 45.25 45.125 -0.0011795589675635
3. 45.125 45.25 45.1875 9.296980478361523e-5
4. 45.1875 45.25 45.21875 -0.0005440173526399188
5. 45.1875 45.21875 45.203125 -0.000861968735520371
6. 45.1875 45.203125 45.1953125 -0.0007030382016190136

After 7 steps, convergent value found.
U(r_0) = -0.0006235390679645697 for V_0/|E| = 45.19140625

Locating Initial V_0.

Initial V_0/|E| = 45.1344062499978
Starting Bisection.

0. 45.133406249997805 45.1344062499978 45.13390624999781 -2.9892413379223334e-6
1. 45.13390624999781 45.1344062499978 45.1341562499978 2.1107456864989785e-6
2. 45.1341562499978 45.1344062499978 45.1342812499978 -4.392596510584749e-7
3. 45.1341562499978 45.1342812499978 45.1342187499978 -1.7142536919430825e-6
4. 45.1341562499978 45.1342187499978 45.1341874999978 -1.076756753729395e-6
5. 45.1341562499978 45.1341874999978 45.1341718

7. 22.125 22.1328125 22.12890625 0.001923297274676015

After 8 steps, convergent value found.
U(r_0) = 0.0019990886817764695 for V_0/|E| = 22.126953125

Locating Initial V_0.

Initial V_0/|E| = 22.23295312500135
Starting Bisection.

0. 22.23195312500135 22.23295312500135 22.23245312500135 -1.151065217664422e-5
1. 22.23195312500135 22.23245312500135 22.232203125001348 -1.8892497601914624e-6
2. 22.232203125001348 22.23245312500135 22.23232812500135 2.921597063429388e-6
3. 22.232203125001348 22.23232812500135 22.232265625001347 5.161608662528406e-7
4. 22.232203125001348 22.232265625001347 22.232234375001347 1.7188749296915765e-6
5. 22.232203125001348 22.232234375001347 22.232218750001348 2.3202358905690496e-6
6. 22.232203125001348 22.232218750001348 22.23221093750135 2.620916545466308e-6
7. 22.232203125001348 22.23221093750135 22.23220703125135 2.7712573627348004e-6
8. 22.232203125001348 22.23220703125135 22.23220507812635 2.8464279555997525e-6
9. 22.232203125001348 22.23220507812635 22.2

2. 13.75 14.0 13.875 -0.001600479987725138
3. 13.75 13.875 13.8125 -0.005200131508124951
4. 13.75 13.8125 13.78125 -0.003408404890536142
5. 13.75 13.78125 13.765625 -0.0025064724816807934
6. 13.75 13.765625 13.7578125 -0.0020539844029373844
7. 13.75 13.7578125 13.75390625 -0.0018273593201994923
8. 13.75 13.75390625 13.751953125 -0.0017139514474902742

After 9 steps, convergent value found.
U(r_0) = -0.001657223666427299 for V_0/|E| = 13.7509765625

Locating Initial V_0.

Initial V_0/|E| = 13.695976562499476
Starting Bisection.

0. 13.694976562499477 13.695976562499476 13.695476562499476 -2.4638197133918703e-5
1. 13.694976562499477 13.695476562499476 13.695226562499476 -9.996411954846734e-6
2. 13.694976562499477 13.695226562499476 13.695101562499477 -2.6751255435139237e-6
3. 13.695101562499477 13.695226562499476 13.695164062499476 9.856135872704877e-7
4. 13.695164062499476 13.695226562499476 13.695195312499475 -8.447628284629432e-7
5. 13.695164062499476 13.695195312499475 13.69517968749

8. 10.164976562499458 10.164980468749459 10.164978515624458 1.1582030828898305e-5
9. 10.164976562499458 10.164978515624458 10.164977539061958 1.1655539514870878e-5
10. 10.164976562499458 10.164977539061958 10.164977050780708 1.1692295604243846e-5
11. 10.164976562499458 10.164977050780708 10.164976806640084 1.1710670056174008e-5

After 12 steps, convergent value found.
U(r_0) = 1.1719859848169686e-5 for V_0/|E| = 10.16497668456977
Locating Initial V_0.

Initial V_0/|E| = 10.0
Starting Bisection.

0. 9.0 10.0 9.5 -0.018134357442420777
1. 9.5 10.0 9.75 0.0007444894108901215
2. 9.75 10.0 9.875 -0.008960164584936646
3. 9.75 9.875 9.8125 -0.013612499938411751
4. 9.75 9.8125 9.78125 -0.011302774601519384
5. 9.75 9.78125 9.765625 -0.010135596839921164
6. 9.75 9.765625 9.7578125 -0.009548914610497266
7. 9.75 9.7578125 9.75390625 -0.009254798335110918
8. 9.75 9.75390625 9.751953125 -0.009107546175559408

After 9 steps, convergent value found.
U(r_0) = -0.00903387156326568 for V_0/|E| = 9.7509765

...


Locating Initial V_0.

Initial V_0/|E| = 85.63181250000417
Starting Bisection.

0. 85.63081250000417 85.63181250000417 85.63131250000417 -3.730122252769844e-6
1. 85.63081250000417 85.63131250000417 85.63106250000416 -9.024526846730826e-7
2. 85.63106250000416 85.63131250000417 85.63118750000416 5.113921109215382e-7
3. 85.63118750000416 85.63131250000417 85.63125000000417 -1.9553095270591307e-7
4. 85.63118750000416 85.63125000000417 85.63121875000417 -5.489917536705312e-7
5. 85.63118750000416 85.63121875000417 85.63120312500416 -3.722616373862044e-7
6. 85.63118750000416 85.63120312500416 85.63119531250416 -2.838975276132359e-7
7. 85.63118750000416 85.63119531250416 85.63119140625416 -2.3971489563992613e-7
8. 85.63118750000416 85.63119140625416 85.63118945312917 -2.1762314770059262e-7
9. 85.63118750000416 85.63118945312917 85.63118847656666 -2.0657764421512918e-7

After 10 steps, convergent value found.
U(r_0) = -2.0105489855765446e-7 for V_0/|E| = 85.6311879882854
Locating Initial V_0.


8. 32.721203124998624 32.721207031248625 32.721205078123624 3.1429894661755598e-6
9. 32.721203124998624 32.721205078123624 32.72120410156113 3.1696461563781674e-6
10. 32.721203124998624 32.72120410156113 32.721203613279876 3.18297499195243e-6

After 11 steps, convergent value found.
U(r_0) = 3.1896378365812674e-6 for V_0/|E| = 32.721203369139246
Locating Initial V_0.

Initial V_0/|E| = 29.0
Starting Bisection.

0. 28.0 29.0 28.5 -0.008247048277169674
1. 28.0 28.5 28.25 -0.0006504926975670213
2. 28.25 28.5 28.375 0.0032362019414481965
3. 28.25 28.375 28.3125 0.0012854325874121868
4. 28.25 28.3125 28.28125 0.002258957689225333
5. 28.25 28.28125 28.265625 0.0027471144138378053
6. 28.25 28.265625 28.2578125 0.002991541763223914
7. 28.25 28.2578125 28.25390625 0.0031138427400433286

After 8 steps, convergent value found.
U(r_0) = 0.00317501506284885 for V_0/|E| = 28.251953125

Locating Initial V_0.

Initial V_0/|E| = 28.457953125001474
Starting Bisection.

0. 28.456953125001473 28.457953125

10. 17.970703125001798 17.970704101564298 17.97070361328305 -1.7305391790838829e-6
11. 17.970703125001798 17.97070361328305 17.970703369142424 -1.719195730674462e-6

After 12 steps, convergent value found.
U(r_0) = -1.7135246368435148e-6 for V_0/|E| = 17.97070324707211
Locating Initial V_0.

Initial V_0/|E| = 17.0
Starting Bisection.

0. 16.0 17.0 16.5 -0.01644419736748503
1. 16.0 16.5 16.25 -0.004425893565664549
2. 16.25 16.5 16.375 0.0018533931881355567
3. 16.375 16.5 16.4375 -0.001309077397985977
4. 16.375 16.4375 16.40625 -0.0028731684247910645
5. 16.375 16.40625 16.390625 -0.0020925466237768716
6. 16.375 16.390625 16.3828125 -0.001701168311711159
7. 16.375 16.3828125 16.37890625 -0.0015052119776482644

After 8 steps, convergent value found.
U(r_0) = -0.001407166973813828 for V_0/|E| = 16.376953125

Locating Initial V_0.

Initial V_0/|E| = 16.32395312500005
Starting Bisection.

0. 16.322953125000048 16.32395312500005 16.32345312500005 -2.3140317649429298e-5
1. 16.322953125000048 16

4. 11.709226562499468 11.709289062499469 11.70925781249947 6.301099883498456e-6
5. 11.709226562499468 11.70925781249947 11.70924218749947 7.354157127786156e-6
6. 11.709226562499468 11.70924218749947 11.709234374999468 7.880688737534617e-6
7. 11.709226562499468 11.709234374999468 11.709230468749467 8.143954237990696e-6
8. 11.709226562499468 11.709230468749467 11.709228515624467 8.275587076301145e-6
9. 11.709226562499468 11.709228515624467 11.709227539061967 8.341402994613344e-6
10. 11.709226562499468 11.709227539061967 11.709227050780719 8.374311875591094e-6
11. 11.709226562499468 11.709227050780719 11.709226806640093 8.390764315892834e-6

After 12 steps, convergent value found.
U(r_0) = 8.398992128518442e-6 for V_0/|E| = 11.709226684569781
Locating Initial V_0.

Initial V_0/|E| = 11.0
Starting Bisection.

0. 10.0 11.0 10.5 -0.0038437535633645587
1. 10.5 11.0 10.75 0.014566609979722808
2. 10.5 10.75 10.625 0.005150156291817447
3. 10.5 10.625 10.5625 0.009804857031973507
4. 10.5 10.5625 


After 9 steps, convergent value found.
U(r_0) = 0.007674160953984788 for V_0/|E| = 8.2509765625

Locating Initial V_0.

Initial V_0/|E| = 8.420976562500018
Starting Bisection.

0. 8.419976562500018 8.420976562500018 8.420476562500017 -4.262937766009868e-5
1. 8.419976562500018 8.420476562500017 8.420226562500018 -2.027771046894155e-5
2. 8.419976562500018 8.420226562500018 8.420101562500019 -9.100826061754945e-6
3. 8.419976562500018 8.420101562500019 8.420039062500019 -3.5121212051739157e-6
4. 8.419976562500018 8.420039062500019 8.420007812500018 -7.177025188473954e-7
5. 8.420007812500018 8.420039062500019 8.420023437500017 6.79521710119798e-7
6. 8.420023437500017 8.420039062500019 8.420031250000019 -1.9092405170416547e-8
7. 8.420023437500017 8.420031250000019 8.420027343750018 -3.683968110972051e-7
8. 8.420023437500017 8.420027343750018 8.420025390625018 -1.937452725551389e-7
9. 8.420023437500017 8.420025390625018 8.420024414062517 -1.0641989109522879e-7
10. 8.420023437500017 8.4200244

### Comparing $U(r)$ vs. $r$

In [243]:
a = 1.3
V_0 = 32.7212

self_50 = System(.1, a, V_0, 50, .001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_50)
U_integration(self_50)

r_50 = collect(self_50.r_0:self_50.∆r:self_50.r_max)

open("data/r_max/U_50.dat", "w") do file
    for i=1:length(r_50)
        println(file, r_50[i], " ", self_50.U[i])
    end
end


self_40 = System(.1, a, V_0, 40, .001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_40)
U_integration(self_40)

r_40 = collect(self_40.r_0:self_40.∆r:self_40.r_max)

open("data/r_max/U_40.dat", "w") do file
    for i=1:length(r_40)
        println(file, r_40[i], " ", self_40.U[i])
    end
end


self_30 = System(.1, a, V_0, 30, .001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_30)
U_integration(self_30)

r_30 = collect(self_30.r_0:self_30.∆r:self_30.r_max)

open("data/r_max/U_30.dat", "w") do file
    for i=1:length(r_30)
        println(file, r_30[i], " ", self_30.U[i])
    end
end


self_20 = System(.1, a, V_0, 20, .001)#System(r_0, a, V_0, r_max, ∆r, ∆V)
initialize(self_20)
U_integration(self_20)

r_20 = collect(self_20.r_0:self_20.∆r:self_20.r_max)

open("data/r_max/U_20.dat", "w") do file
    for i=1:length(r_20)
        println(file, r_20[i], " ", self_20.U[i])
    end
end


## Exporting $U(r)$ for true value of $a$

In [289]:
a, V_0 = 1.258, 34.8307
self_0 = System(.1, a, V_0, 30, 1e-3)

r_values = collect(self_0.r_0:self_0.∆r:self_0.r_max)

initialize(self_0)
U_integration(self_0)

norm_0= calculate_norm(self_0)

open("data/U_vs_r.dat", "w") do file
    for i=1:length(r_values)
        println(file, r_values[i], " ", self_0.U[i]/sqrt(norm))
    end
end

## Testing Impact of initial conditions

In [310]:
a, V_0 = 1.258, 34.8307
self = System(.1, a, V_0, 30, 1e-3)

r_values = collect(self.r_0:self.∆r:self.r_max)

initialize(self)

#set value of U at r = r_max first
self.U[self.n_total] = .001
#set corresponding vaue of phi_0
self.phi_0 = self.U[self.n_total] * (1-((self.∆r^2) * f(self, self.r_max) / 12))

#set value of U at r = r_max - ∆r next
self.U[self.n_total-1] = 100000000000000
#set corresponding vaue of phi_1
self.phi_1 = self.U[self.n_total-1] * (1-((self.∆r^2) * f(self, self.r_max - self.∆r) / 12))

U_integration(self)